## Create urban built-up and population density gradients for multiple cities

In [66]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point
from shapely import wkt
import plotly.express as px
import os
import gdal
from rasterio.mask import mask
import plotly.graph_objects as go
import numpy as np
import rasterio
import pylab as pl
from rasterio.warp import calculate_default_transform, reproject, Resampling
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
#dat = pd.read_csv("..\\GHSL_NewVersion\\GHSL_UCDB_new.csv")

In [153]:
def return_point(city):
    
    cities = {'Accra, Ghana': (-0.203739, 5.550775),
          'Buenos Aires, Argentina': (-58.371983, -34.608481), 
          'Paris, France' : (2.239555, 48.891194), 
          'Chicago, USA' : (-87.635393, 41.880952), 
          'Dhaka, Bangladesh': (90.417744, 23.729136), 
          'Shanghai, China' : (121.472155, 31.236955)}
    
    #string = city.split(',')[0]
    #dat = pd.read_csv("..\\GHSL_NewVersion\\GHSL_UCDB_new.csv")
    #df = dat[dat.UC_NM_MN == string]
    
    point = Point(cities[city])
    
    return point
    #import osmnx as ox
    
    #x = ox.gdf_from_place(city)['geometry'].iloc[0]
    
    #if x.geom_type == 'Point':
    #    return x
    #elif x.geom_type == 'Polygon':
    #    return x.centroid

In [3]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio accepts them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [184]:
def get_buffer_vars(city):
    
    point = return_point(city)
    buff_250 = point.buffer(0.005, 128)
    buff_500 = point.buffer(0.01, 128)
    dough_500 = buff_500.difference(buff_250)
    buff_1000 = point.buffer(0.015, 128)
    dough_1000 = buff_1000.difference(buff_500)
    buff_1500 = point.buffer(0.02, 128)
    dough_1500 = buff_1500.difference(buff_1000)
    buff_2000 = point.buffer(0.025, 128)
    dough_2000 = buff_2000.difference(buff_1500)
    buff_2500 = point.buffer(0.03, 128)
    dough_2500 = buff_2500.difference(buff_2000)
    
    return buff_250, dough_500, dough_1000, dough_1500, dough_2000, dough_2500

In [154]:
def convert_geom_to_shp(shapely_polygon, city, out_name_postfix):
    string = city.split(',')[0]
    
    df = pd.DataFrame(
    {'City': [string],
     'geometry': [wkt.dumps(shapely_polygon)]})
    
    df['geometry'] = df['geometry'].apply(wkt.loads)
    
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    gdf.crs = {'init' : 'epsg:4326'}
    
    path = "M:\Gaurav\GPSUR\Data\Analysis"
    
    gdf.to_file(path+'\\shapefiles\\{a}_{b}.shp'.format(a=string, b=out_name_postfix))
    
    return path+'\\shapefiles\\{a}_{b}.shp'.format(a=string, b=out_name_postfix)

In [6]:
def reproject_raster(source_path, destination_path):

    dst_crs = 'EPSG:4326'
    
    try:
        with rasterio.open(source_path, CHECK_DISK_FREE_SPACE =False) as src:
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })

            with rasterio.open(destination_path, 'w',**kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)
    except rasterio._err.CPLE_FileIOError:
        print('File too big! Cant open')
    except rasterio._base.RasterioIOError:
        print('Raster file empty!')
        

In [27]:
def get_clipped_density(shpfile_path, raster_path):
    """
    Function to clip the population gridded data by polygons from datasets.
    """
    shp = gpd.read_file(shpfile_path)
    shp.crs = {'init':'epsg:4326'}
    shp = shp.to_crs(epsg=4326)
    
    coords = getFeatures(shp)
    
    new = rasterio.open(raster_path)
    
    try:
        out_img, out_transform = mask(dataset=new, shapes=coords, crop=True)
        new_out = out_img[out_img > 0]
        print('Overlap found!')
        return new_out.sum()/len(new_out)
        
    except ValueError:
        print('No Overlap between raster and polygon.')
        return

In [88]:
def get_clipped_pop_density(shpfile_path, raster_path):
    """
    Function to clip the population gridded data by polygons from datasets.
    """
    shp = gpd.read_file(shpfile_path)
    shp.crs = {'init':'epsg:4326'}
    shp = shp.to_crs(epsg=4326)
    
    coords = getFeatures(shp)
    
    new = rasterio.open(raster_path)
    
    try:
        out_img, out_transform = mask(dataset=new, shapes=coords, crop=True)
        new_out = out_img[out_img > 0]
        print('Overlap found!')
        return new_out.sum()
        
    except ValueError:
        print('No Overlap between raster and polygon.')
        return

In [180]:
def main(city):
    
    point = return_point(city)
    
    path = r"M:\Gaurav\GPSUR\Data\GHSL_builtup_250m"
    
    temp = os.listdir(path)
    
    buff_250, buff_500, buff_1000, buff_1500, buff_2000, buff_2500 = get_buffer_vars(city)
    
    dic = {'250m':buff_250, '500m':buff_500, '1000m':buff_1000, '1500m':buff_1500, '2000m':buff_2000, '2500m':buff_2500}
    
    shp_lis = []
    for res in dic.keys():  
        shp_lis.append(convert_geom_to_shp(dic[res], city, res))
    
    sum_dic = {}
    
    for i in [1990, 2000, 2014]:
        for j in temp:
            if str(i) in j.split('_'):
                path_ext = path+"\\"+j
                print(path_ext)
                #TIF_lis = [i for i in os.listdir(path_ext) if i.endswith('.TIF')]
        
                #for k in TIF_lis:
                 #   reproject_raster(path_ext+"\\"+k, path_ext+"\\GHS_builtup_"+str(i)+"_reprojected_4326"+k.split('_')[-1].split('.')[0]+'.tif')
            
                tif_lis = [i for i in os.listdir(path_ext) if i.endswith('.tif')]
        
                sum_lis = []

                for shp in shp_lis:
                    for ras in tif_lis:
                        sum_lis.append(get_clipped_density(shp, path_ext+"\\"+ras))
                sum_dic[i] = sum_lis
                
    return sum_dic

In [197]:
#master = {}
#for i in cities:
#   string = i.split(',')[0]
#    master[string] = main(i)

In [186]:
master_non_1 = {}

for i in master.keys():
    temp = {}
    for j in master[i].keys():
        temp[j] = [round(m, 2) for m in master[i][j] if (m) and (np.isnan(m) != True)]
        
    master_non_1[i] = temp 

In [188]:
for year in [1990, 2000, 2014]:
    fig = go.Figure()
    for key in master_non_1.keys():
        x = [250, 500, 1000, 1500, 2000, 2500]
        
        fig.add_trace(go.Scatter(x=x, y=master_non_1[key][year],
                     mode='lines+markers', text=master_non_1[key][year],  name='Avg Built-up for {}'.format(key)))
        
    fig.update_layout(
        title={
            'text': "Share of built-up in {} at different buffer levels".format(year),
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

    fig.update_layout(xaxis_title='Buffer Distance (in m)',
                       yaxis_title='Share of Built-up')
    fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_xaxes(range=[200,2600])
    #fig.update_yaxes(range=[0,100])
    
    fig.show()

In [72]:

#for i in di2.keys():
#    x = [500, 1000, 1500, 2000]
#    fig = go.Figure(data=go.Scatter(x=x, y=di2[i], mode='lines+markers', text=di2[i],  name='% Built-up'))
#
#    fig.update_layout(
#        title={
#            'text': "Share of built-up in {} at different buffer levels".format(i),
#            'y':0.9,
#            'x':0.5,
#            'xanchor': 'center',
#            'yanchor': 'top'})
#
#    fig.update_layout(xaxis_title='Buffer Distance (in m)',
#                       yaxis_title='% Share of Built-up')
#    fig.update_traces(hoverinfo='text+name', mode='lines+markers')
#    fig.update_xaxes(range=[250,2100])
#    fig.update_yaxes(range=[0,100])
#    
#    fig.show()

## Population

In [191]:
def main_pop(city):
    
    point = return_point(city)
    
    path = r"M:\Gaurav\GPSUR\Data\GHSL Pop"
    
    temp = os.listdir(path)
    
    buff_250, buff_500, buff_1000, buff_1500, buff_2000, buff_2500 = get_buffer_vars(city)
    
    dic = {'250m':buff_250, '500m':buff_500, '1000m':buff_1000, '1500m':buff_1500, '2000m':buff_2000, '2500m':buff_2500}
    
    shp_lis = []
    for res in dic.keys():  
        shp_lis.append(convert_geom_to_shp(dic[res], city, res))
    
    sum_dic = {}
    
    for i in [1990, 2000, 2015]:
        for j in temp:
            if str(i) in j.split('_'):
                path_ext = path+"\\"+j
                print(path_ext)
                #TIF_lis = [i for i in os.listdir(path_ext) if i.endswith('.TIF')]
        
                #for k in TIF_lis:
                #    reproject_raster(path_ext+"\\"+k, path_ext+"\\GHS_Pop_"+str(i)+"_reprojected_4326"+k.split('_')[-1].split('.')[0]+'.tif')
            
                tif_lis = [i for i in os.listdir(path_ext) if i.endswith('.tif')]
        
                sum_lis = []

                for shp in shp_lis:
                    for ras in tif_lis:
                        sum_lis.append(get_clipped_pop_density(shp, path_ext+"\\"+ras))
                sum_dic[i] = sum_lis
                
    return sum_dic

In [196]:
#master_pop = {}
#for i in cities:
#    string = i.split(',')[0]
#    master_pop[string] = main_pop(i)

In [193]:
master_pop_non = {}

for i in master_pop.keys():
    temp = {}
    for j in master_pop[i].keys():
        temp[j] = [round(m, 2) for m in master_pop[i][j] if (m) and (np.isnan(m) != True)]
        
    master_pop_non[i] = temp

In [194]:
master_pop_non

{'Accra': {1990: [7524.57, 6270.68, 25522.97, 9800.63, 32982.24, 31598.49],
  2000: [10788.07, 10078.44, 37349.93, 14291.14, 49097.28, 46438.19],
  2015: [17594.28, 16436.94, 60914.05, 23307.43, 80077.76, 75736.12]},
 'Buenos Aires': {1990: [12375.56,
   24475.08,
   54852.13,
   45096.45,
   65797.36,
   76081.41],
  2000: [13256.55, 27427.34, 57841.46, 48212.91, 70944.25, 81512.22],
  2015: [13925.37, 29050.15, 60787.96, 51016.2, 75398.81, 85833.41]},
 'Paris': {1990: [12895.07, 12428.13, 49917.82, 40275.42, 51414.97, 47294.35],
  2000: [13933.93, 14121.56, 54880.95, 42662.77, 56946.59, 52173.61],
  2015: [15669.23, 17284.11, 62830.72, 46885.21, 65355.53, 59959.26]},
 'Chicago': {1990: [4052.28, 22349.96, 5073.36, 19736.21],
  2000: [4451.9, 24548.06, 5591.55, 21687.18],
  2015: [4163.24, 22956.41, 5229.0, 20281.57]},
 'Dhaka': {1990: [17936.62,
   68284.7,
   177269.74,
   127052.34,
   329281.53,
   175810.31],
  2000: [27795.23, 101372.73, 261196.87, 187336.8, 446012.95, 262205.11

In [195]:
for year in [1990, 2000, 2015]:
    fig = go.Figure()
    for key in master_pop_non.keys():
        x = [250, 500, 1000, 1500, 2000, 2500]
        
        fig.add_trace(go.Scatter(x=x, y=master_pop_non[key][year],
                     mode='lines+markers', text=master_pop_non[key][year],  name='{}'.format(key)))
        
    fig.update_layout(
        title={
            'text': "Population in {} at different buffer levels".format(year),
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

    fig.update_layout(xaxis_title='Buffer Distance (in m)',
                       yaxis_title='Population')
    fig.update_traces(hoverinfo='text+name', mode='lines+markers')
    fig.update_xaxes(range=[250,2100])
    #fig.update_yaxes(range=[0,28000000])
    
    fig.show()